## I.1. Install

In [ ]:
# # GOOGLE Colab ONLY : Mount GOOGLE Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
# GOOGLE Colab ONLY : Get & store the default work path
user_path = os.path.join(os.getcwd(), "drive", "MyDrive", "Colab Notebooks")

# LOCAL : Get & store the current working path
# user_path = os.path.dirname(os.path.abspath("NoNeedFilename"))

print(user_path)

In [ ]:
# requirements_filepath = os.path.join(user_path, "requirements.txt")
# !pip install -r {requirements_filepath}

## I.2. Libraries Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Machine Learning
# import keras
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import Adam, Nadam, SGD, RMSprop
from sklearn.metrics import confusion_matrix, classification_report, f1_score

from keras.applications import EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3

## I.4. Image Processing

### I.4.2. Set Image Folder Path

In [ ]:
# GOOGLE Colab ONLY
train_dir = os.path.join(user_path, "IMAGE_SPLIT", "train")
val_dir = os.path.join(user_path, "IMAGE_SPLIT", "val")
test_dir = os.path.join(user_path, "IMAGE_SPLIT", "test")

# LOCAL
# train_dir = r"C:\WORKSPACES\DATA\ML_DL\IMAGE_SPLIT\train"
# val_dir = r"C:\WORKSPACES\DATA\ML_DL\IMAGE_SPLIT\val"
# test_dir = r"C:\WORKSPACES\DATA\ML_DL\IMAGE_SPLIT\test"

### I.4.3. Data augmentation techniques

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = [0.9, 1.25],
    brightness_range = [0.5, 1.5],
    horizontal_flip = True,
    vertical_flip = True
)

val_datagen = ImageDataGenerator(
    rescale = 1./255)

test_datagen = ImageDataGenerator(
    rescale = 1./255)

### I.4.4. Setup Data generators

In [ ]:
# Variables
batch_size = 32
height = 240
width = 240
image_shape = (height, width) # images of size 240x240 px with 3 color channels RGB automatically

train_generator = train_datagen.flow_from_directory(
    train_dir,
    class_mode = "categorical",
    target_size = image_shape,
    batch_size = batch_size
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    class_mode = "categorical",
    target_size = image_shape,
    batch_size = batch_size
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    class_mode = "categorical",
    target_size = image_shape,
    batch_size = batch_size,
    shuffle = False
)

## II.1. Define variables and parameters

In [ ]:
from keras.metrics import Precision, Recall, F1Score, AUC

# Model to use for transfer learning
model_name = "EfficientNetV2B1"
ModelNameFunc = globals()[model_name]

# Work path
folder_name = f"{model_name}_L1N128-DO5"
work_path = os.path.join(user_path, folder_name)

# Top Architecture for the new model
l1_neurons = 128                 # Number of neurons on layer 1 (deeper)
l1_activation = "relu"
dropout = 0.5                   # 0.2 - 0.5
l2_neurons = 32                 # Number of neurons on layer 2
l2_activation = "relu"
output_neurons = 3              # This layer will have 3 neurons or units fully connected. In line with our 3 classes.
output_activation = "softmax"

# Metrics (parameter average = "macro" by default )
precision = Precision(name = "Precision")
recall = Recall(name = "Recall")
# auc = AUC(name = "AUC")

# TRAIN : model.compile
compile_optimizer = "Adam"                  # Test : Nadam, RMSprop, SGD
compile_loss = "categorical_crossentropy"   # More than 2 classes, so categorical
compile_metrics = ["accuracy", precision, recall]   # auc

# TRAIN : model.fit
epochs = 30
verbose = 1

# FINE TUNE : model.compile
ft_compile_optimizer = Adam(learning_rate = 1e-5)
# ft_compile_optimizer : Very low learning rate. Test : Nadam, RMSprop, SGD
ft_compile_loss = "categorical_crossentropy"
ft_compile_metrics = ["accuracy", precision, recall]    # auc

# FINE TUNE : model.fit
ft_epochs = 15
ft_verbose = 1

# SAVE : File Path .h5
fileh5_path = os.path.join(work_path, f"{folder_name}_H5_Model", f"{folder_name}_TRAIN_ManualSave.h5")
ft_fileh5_path = os.path.join(work_path, f"{folder_name}_H5_Model", f"{folder_name}_FT_ManualSave.h5")

## II.2. Define optimizations

### II.2.1. Train optimizations (callbacks)

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, CSVLogger

# Define the early stopping
early_stopping = EarlyStopping(
    monitor = "val_loss",        # Metric to monitor
    patience = 5,                # Number of epochs with no improvement  (around 10% of nb of epochs)
    verbose = 1,                 # Verbosity mode (0, 1, or 2)
    restore_best_weights = True  # Whether to restore model weights to the best epoch
)

# Define the learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor = "val_loss",       # Metric to monitor
    factor = 0.2,               # Factor by which to reduce the learning rate
    patience = 5,               # Number of epochs with no improvement (default = 10)
    verbose = 1,                # Verbosity mode (0, 1, or 2)
    min_lr = 1e-6               # Minimum learning rate
)

# Define the model checkpoint callback
checkpoint = ModelCheckpoint(
    # filepath = os.path.join(work_path, f"{model_name}_TRAIN_CheckPoint"), # to use if don't wanna save to a .h5 or .keras file
    filepath = os.path.join(work_path, f"{folder_name}_H5_Model", f"{folder_name}_TRAIN_MCPBest.h5"),
    monitor = "val_loss",       # Metric to monitor
    save_freq = "epoch",        # Default = epoch
    save_weights_only = False,  # Default = False
    save_best_only = True,      # Save only if the monitored metric improves
    mode = "min",               # "min" if monitoring loss, "max" if monitoring accuracy
    verbose = 1                 # Verbosity mode (0, 1, or 2)
)

# Define TensorBoard Logs
log_dir = os.path.join(work_path, f"{folder_name}_LOGS_TRAIN")
tensorboard = TensorBoard(
    log_dir = log_dir,
    write_graph = True,
    write_images = True,
    update_freq = "epoch"
)

# Define CSVLogger
CSV_Log_filename = os.path.join(work_path, f"{folder_name}_TRAIN.log")
CSV_Logger = CSVLogger(
    CSV_Log_filename,
    separator = "|",
    append = False
)



# Define the callbacks list
callbacks_list = [early_stopping, lr_scheduler, checkpoint, tensorboard, CSV_Logger]

### II.2.2. Fine Tune Optimization

In [ ]:
# Define the FT Early stopping
ft_early_stopping = EarlyStopping(
    monitor = "val_loss",
    patience = 3,
    verbose = 1,                 # Verbosity mode (0, 1, or 2)
    restore_best_weights = True  # Whether to restore model weights to the best epoch
)

# Define the FT Learning rate scheduler
ft_lr_scheduler = ReduceLROnPlateau(
    monitor = "val_loss",  # Metric to monitor
    factor = 0.2,          # Factor by which to reduce the learning rate
    patience = 3,          # Number of epochs with no improvement (default = 10)
    verbose = 1,           # Verbosity mode (0, 1, or 2)
    min_lr = 1e-6          # Minimum learning rate
)

# Define the FT checkpoint callback
ft_checkpoint = ModelCheckpoint(
    # filepath = os.path.join(work_path, f"{model_name}_FT_CheckPoint"), # to use if don't wanna save to a .h5 or .keras file
    filepath = os.path.join(work_path, f"{folder_name}_H5_Model", f"{folder_name}_FT_MCPBest.h5"),
    monitor = "val_loss",       # Metric to monitor
    save_freq = "epoch",        # Default = epoch
    save_weights_only = False,  # Default = False
    save_best_only = True,      # Save only if the monitored metric improves
    mode = "min",               # "min" if monitoring loss, "max" if monitoring accuracy
    verbose = 1                 # Verbosity mode (0, 1, or 2)
)

# Define the FT TensorBoard Logs
ft_log_dir = os.path.join(work_path, f"{folder_name}_LOGS_FT")
ft_tensorboard = TensorBoard(
    log_dir = ft_log_dir,
    write_graph=True,
    write_images=True,
    update_freq = "epoch"
)

# Define the FT CSVLogger
ft_CSV_Log_filename = os.path.join(work_path, f"{folder_name}FT.log")
ft_CSV_Logger = CSVLogger(
    ft_CSV_Log_filename,
    separator = "|",
    append = False
)

# Define the FT callbacks list
ft_callbacks_list = [ft_early_stopping, ft_lr_scheduler, ft_checkpoint, ft_tensorboard, ft_CSV_Logger]

## II.3. Pre train the model

### II.3.1. Instantiate a base model with pre-trained model and weights

In [ ]:
# First instantiate a base model with pre-trained weights
base_model = ModelNameFunc(
    include_top = False,
    weights = "imagenet",
    include_preprocessing = False, # ONLY for V2 model, otherwise put it as comment.
    input_shape = (height, width, output_neurons)
)

In [ ]:
display(base_model.input)

### II.3.2. Define the new architecture on top

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x) # Pooling
x = Dense(l1_neurons, activation = l1_activation)(x)
x = Dropout(dropout)(x)
x = Dense(l2_neurons, activation = l2_activation)(x)
x = BatchNormalization()(x)

predictions = Dense(output_neurons, activation = output_activation)(x)

model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
print(model.input_shape)

### II.3.4. Compile the model

In [ ]:
model.compile(
    optimizer = compile_optimizer,
    loss = compile_loss,
    metrics = compile_metrics
)

### II.3.5. Train the model
It's called "history" because it essentially keeps a historical record of various metrics and data related to how the model performed during training.

In [ ]:
history = model.fit(
    x = train_generator,
    steps_per_epoch = train_generator.samples // batch_size + 1,
    validation_data = val_generator,
    validation_steps = val_generator.samples // batch_size + 1,
    epochs = epochs,
    verbose = verbose,
    callbacks = callbacks_list
)

# Save the model
model.save(fileh5_path)

### II.3.7. Evaluate Pre trained model

In [ ]:
# Predict the values from the Test Dataset
Y_pred = model.predict(test_generator, steps = test_generator.samples // batch_size + 1)

# Convert predictions classes from one hot vectors 
Y_pred_classes = np.argmax(Y_pred, axis = 1)

In [ ]:
# Evaluate the model on the Test Dataset using `evaluate`
# f1 = F1Score(test_generator.classes, Y_pred_classes)
f1 = f1_score(test_generator.classes, Y_pred_classes, average = "macro")
# f1 = (2 * (precision * recall) / (precision + recall))
print("F1 Score : ", f1)
print("Evaluate on Test Data")
results = model.evaluate(test_generator, batch_size = batch_size)

### II.3.8. Plot the Accuracy and the loss graphs

In [ ]:
# Plot the Loss graph
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Val"], loc = "upper left")
plt.show()

In [ ]:
# Plot the Accuracy graph
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Val"], loc = "lower left")
plt.show()

### II.3.8. Trained Model : Confusion Matrix

In [ ]:
# # Predict the values from the Test Dataset
# Y_pred = model.predict(test_generator, steps = test_generator.samples // batch_size + 1)

# # Convert predictions classes from one hot vectors 
# Y_pred_classes = np.argmax(Y_pred, axis = 1)

# Convert validation observations from one hot vectors
# Y_true = np.argmax(test_generator.classes, axis = 1)
# OR
Y_true = test_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(Y_true, Y_pred_classes)

# Print the confusion matrix
print("Confusion Matrix")
print(cm)

# Retrieve class names (label)
names_from_dir = sorted(os.listdir(val_dir))
dict(enumerate(names_from_dir))
#OR
class_names = list(val_generator.class_indices.keys())
# Check if the both are similar
print(names_from_dir == class_names)

# Print the classification report
print("Classification Report")
print(classification_report(Y_true, Y_pred_classes, target_names = class_names, digits = 4))


### II.3.9. Confusion Matrix Function

In [ ]:
def plot_confusion_matrix(
    cm,
    classes,
    normalize = False,
    title = "Confusion matrix",
    cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype("float") / cm.sum(axis =1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation = "nearest", cmap = cmap)
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

### II.3.10. Trained Model : Confusion Matrix Graph

In [ ]:
plot_confusion_matrix(
    cm,
    class_names,
    normalize = True,
    title = "Confusion matrix",
    cmap = plt.cm.Blues
    )

## II.4. Fine Tuning
This requires unfreezing the base model and then recompiling the model (necessary for the changes to take effect), and then resuming training.

### II.4.1. Unfreeze the base model

In [ ]:
# Unfreeze the base model
base_model.trainable = True

### II.4.2. Re compile the model with a very low learning rate

In [ ]:
# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(
    optimizer = ft_compile_optimizer,
    loss = ft_compile_loss,
    metrics = ft_compile_metrics
)

### II.4.3. Retraining the model

In [ ]:
# Resume training
ft_history = model.fit(
    x = train_generator,
    steps_per_epoch = train_generator.samples // batch_size + 1,
    validation_data = val_generator,
    validation_steps = val_generator.samples // batch_size + 1,
    epochs = ft_epochs,
    verbose = ft_verbose,
    callbacks = ft_callbacks_list
)

# Save the Fine tuned model
model.save(ft_fileh5_path)

### II.4.4. Evaluation Fine Tune Model

In [ ]:
# Evaluate the model on the Test Dataset using `evaluate`
# f1 = F1Score(test_generator.classes, Y_pred_classes)
ft_f1 = f1_score(test_generator.classes, Y_pred_classes, average = "macro")
print("F1 Score : ", ft_f1)
print("Evaluate on Test Data")
ft_results = model.evaluate(test_generator, batch_size = batch_size)

In [ ]:
# Plot the Loss graph
plt.plot(ft_history.history["loss"])
plt.plot(ft_history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["Train", "Val"], loc = "upper left")
plt.show()

In [ ]:
# Plot the Accuracy graph
plt.plot(ft_history.history["accuracy"])
plt.plot(ft_history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Val"], loc = "lower left")
plt.show()

### II.4.5. Fine Tuned Model : Confusion Matrix

In [ ]:
# # Predict the values from the validation dataset
# Y_pred = model.predict(test_generator, steps = test_generator.samples // batch_size + 1)

# # Convert predictions classes from one hot vectors 
# Y_pred_classes = np.argmax(Y_pred, axis = 1)

# Convert validation observations from one hot vectors
# Y_true = np.argmax(test_generator.classes, axis = 1)
# OR
Y_true = test_generator.classes

# Compute the confusion matrix
ft_cm = confusion_matrix(Y_true, Y_pred_classes)

# Print the confusion matrix
print("Confusion Matrix")
print(ft_cm)

# Retrieve class names (label)
names_from_dir = sorted(os.listdir(val_dir))
dict(enumerate(names_from_dir))
#OR
class_names = list(val_generator.class_indices.keys())
# Check if the both are similar
print(names_from_dir == class_names)

# Print the classification report
print("Classification Report")
print(classification_report(Y_true, Y_pred_classes, target_names = class_names, digits = 4))

### II.4.5. Fine Tuned Model : Confusion Matrix Graph

In [ ]:
plot_confusion_matrix(ft_cm,
                      class_names,
                      normalize = True,
                      title = "Confusion matrix",
                      cmap = plt.cm.Blues
                      )

### II.4.6. Some metrics of Fine Tune Model
```py
Normalement pas besoin car keras.metrics nous donne les infos avec le `results = model.evaluate(test_generator, batch_size = batch_size)`. (average = "macro" par défaut)
```

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calcul de la précision
precision = precision_score(test_generator.classes, Y_pred_classes, average='macro') # average macro, calcul individuel par classe attribut le même poids à chaque classe
# Calcul du recall
recall = recall_score(test_generator.classes, Y_pred_classes, average='macro')

# Calcul du F1 score
f1 = f1_score(test_generator.classes, Y_pred_classes, average='macro')

# Affichage des résultats
print("Précision :", precision)
print("Recall :", recall)
print("F1 score :", f1)

# III. READ LOGS

## III.1. TRAIN

In [ ]:
%load_ext tensorboard
%tensorboard --logdir log_dir

## III.2. FINE TUNE

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ft_log_dir